In [1]:
import pandas as pd
import requests
import re
import numpy as np

In [31]:
def store_info(x):
    url = f'https://api-extern.systembolaget.se/sb-api-ecommerce/v1/site/stores/{x}'
    headers = {'ocp-apim-subscription-key': 'cfc702aed3094c86b92d6d4ff7a54c84' }
    r = requests.get(url, headers=headers)
    assert r.status_code == 200, r.text

    
    df = pd.json_normalize(r.json()['storeStocks'])[['store.siteId', 'store.alias','store.isOpen','store.address','store.city','store.openingHours','store.position.latitude','store.position.longitude','stockBalance.stock']]
    return df


def search(search_query):
    search_url = f'https://api-extern.systembolaget.se/sb-api-ecommerce/v1/productsearch/search?'
    headers = {'ocp-apim-subscription-key': 'cfc702aed3094c86b92d6d4ff7a54c84' }
    data = {
        'sortBy':'Score',
        'sortDirection': 'Ascending',
        'size': 30,
        'page': 1,
        'textQuery': search_query,
        'isInDepotStockForFastDelivery': 'false'
    }
    r = requests.get(search_url, headers=headers, params=data)

    assert r.status_code == 200, r.text
    df = pd.DataFrame(r.json()['products'])[['productId', 'productNumber', 'productNameBold', 'productNameThin','alcoholPercentage','volume','price','country','categoryLevel3','customCategoryTitle','taste','images']]
    df['image'] = df['images'].apply(lambda x: x[0]['imageUrl'] if type(x) == list and len(x) > 0 else None)

    return df

In [32]:
df = search('rom')

In [36]:
import numpy as np
def distance(store_x, store_y, client_x, client_y):
    return np.sqrt((store_x - client_x)**2 + (store_y - client_y)**2)

In [39]:
df1

,store.siteId,store.alias,store.isOpen,store.address,store.city,store.openingHours,store.position.latitude,store.position.longitude,stockBalance.stock
0,0102,Fältöversten,True,Karlaplan 13,STOCKHOLM,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.338810,18.090880,14
1,0104,None,True,Nybrogatan 47,STOCKHOLM,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.337147,18.078999,9
2,0106,Garnisonen,True,Karlavägen 100 A,STOCKHOLM,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.336104,18.096200,7
3,0107,Norra Djurgårdsstaden,True,Bobergsgatan 50,STOCKHOLM,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.358524,18.091919,13
4,0113,"Sergel, ingång T-Centralen",True,Drottninggatan 45,STOCKHOLM,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.332488,18.062269,18
...,...,...,...,...,...,...,...,...,...
327,2504,None,True,Storgatan 65,PITEÅ,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.317694,21.477423,20
328,2505,None,True,Drottninggatan 8b,BODEN,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.823467,21.685058,15
329,2506,None,True,Norra Esplanaden 8,HAPARANDA,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.840784,24.133352,56
330,2507,None,True,Geologgatan 7 B,KIRUNA,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",67.854328,20.227756,10


In [41]:
df1 = store_info(df.iloc[0]['productId'])

df1['distance'] = df1.apply(lambda x: distance(x['store.position.latitude'], x['store.position.longitude'], 59.238810, 18.190880),axis=1)

In [58]:
from datetime import datetime
now_day = datetime.now().strftime('%Y-%m-%d')
now_time = datetime.now().strftime('%H:%M:%S')

In [69]:
df.columns

Index(['productId', 'productNumber', 'productNameBold', 'productNameThin',
       'alcoholPercentage', 'volume', 'price', 'country', 'categoryLevel3',
       'customCategoryTitle', 'taste', 'images', 'image'],
      dtype='object')

In [83]:
now = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
def time_until_close(x):
    date = pd.to_datetime(x[1]['date']).strftime('%Y-%m-%d')
    if date == now_day:
        if pd.to_datetime(x[0]['openFrom']) < datetime.now() < pd.to_datetime(x[0]['openFrom']):
            return str(pd.to_datetime(x[0]['openTo']) - datetime.now())


df1['time_until_close'] = df1['store.openingHours'].apply(time_until_close)
df1['open_and_in_stock'] = df1['time_until_close'] | df1['stockBalance.stock'] > 0

In [87]:
df1[df1['open_and_in_stock'] == False].sort_values('distance', ascending=True)

,store.siteId,store.alias,store.isOpen,store.address,store.city,store.openingHours,store.position.latitude,store.position.longitude,stockBalance.stock,distance,time_until_close,,open_and_in_stock
54,0221,None,True,Marknadsgränd 29,TYRESÖ,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.243458,18.230902,21,0.040291,None,True,False
60,0227,Skogås Centrum,True,Skogåsplan 9 bv,SKOGÅS,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.217587,18.153991,18,0.042558,None,True,False
76,0245,Haninge Söderby,True,Österport 2,HANINGE,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.181270,18.158600,18,0.065976,None,True,False
53,0220,None,True,Forum,NACKA,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.310138,18.163900,29,0.076260,None,True,False
55,0222,Haninge Centrum,True,Drakslingan 12,HANINGE,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",59.170344,18.139923,17,0.085348,None,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2503,Storheden,True,Besiktningsvägen 19 A,LULEÅ,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.619001,22.046209,16,7.454556,None,True,False
325,2501,None,True,Storgatan 61,LULEÅ,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.585008,22.158721,17,7.484517,None,True,False
331,2520,Skutviken,True,Delfingatan 2,LULEÅ,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",65.595622,22.148602,10,7.488165,None,True,False
330,2507,None,True,Geologgatan 7 B,KIRUNA,"[{'date': '2022-06-18T00:00:00', 'openFrom': '...",67.854328,20.227756,10,8.853023,None,True,False
